In [4]:
import sys
print(sys.version)

3.6.8 |Anaconda custom (64-bit)| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [5]:
import socket
import pandas as pd
import random
import os
import json
import numpy as np
from pyspark.rdd import RDD
from pydoop import hdfs
from pyspark.sql import SparkSession,Row
from pyspark import TaskContext
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from RSP import RSPDM,RSPdataset#*
from time import *
from datetime import datetime
from sklearn.linear_model import SGDClassifier
from multiprocessing.pool import ThreadPool

In [6]:
spark = SparkSession.builder.appName(
    "SVM-EL-PY-Class2"
).config(
    "spark.executor.memory","64g"
).config(
    "spark.executor.cores","8"
).config(
    "spark.dynamicAllocation.maxExecutors", "20"
).config(
    "spark.yarn.executor.memoryOverhead", str(2239)
).config(
    "spark.kryoserializer.buffer.max", "1024m"
).config(
    "spark.driver.memory","32g"
).config(
    "spark.locality.wait","15"
).config(
    "yarn.nodemanager.vmem-check-enabled","false"
).config(
    "yarn.nodemanager.pmem-check-enabled","false"
).getOrCreate()

In [7]:
spark.version

'2.4.0.cloudera2'

In [8]:
bspfilepath = "/user/luokaijing/classification_BSP/raw_kaggle/train.csv"
rawDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(bspfilepath)

In [9]:
rawDF.printSchema()

root
 |-- lgbmcpu: double (nullable = true)
 |-- lgbmgpu: double (nullable = true)
 |-- xgb: double (nullable = true)
 |-- catgpu: double (nullable = true)
 |-- catcpu: double (nullable = true)
 |-- target: integer (nullable = true)
 |-- histcpu: double (nullable = true)
 |-- lgbmseed: double (nullable = true)
 |-- lgbmseed2: double (nullable = true)
 |-- lgbmseed3: double (nullable = true)
 |-- lgbmseed4: double (nullable = true)
 |-- voting_5cat: double (nullable = true)
 |-- voting_7cat: double (nullable = true)
 |-- voting_5xgb: double (nullable = true)
 |-- catseed: double (nullable = true)
 |-- lgbmgpu_seed: double (nullable = true)
 |-- lgbm1_seed: double (nullable = true)
 |-- lgbm2_seed: double (nullable = true)
 |-- lgbm3_seed: double (nullable = true)
 |-- xgb_seed: double (nullable = true)
 |-- hist_seed: double (nullable = true)
 |-- xgb_op: double (nullable = true)
 |-- lgbm_k: double (nullable = true)
 |-- lgbm2_k: double (nullable = true)
 |-- cat_k: double (nullable = 

In [10]:
#core:convertion
from pyspark.sql.functions import array
from pyspark.sql.functions import col
cols1 = ["lgbmcpu", "lgbmgpu", "xgb", "catgpu", "catcpu", "histcpu", "lgbmseed", "lgbmseed2", "lgbmseed3", "lgbmseed4", "voting_5cat", "voting_7cat", "voting_5xgb", "catseed", "lgbmgpu_seed", "lgbm1_seed", "lgbm2_seed", "lgbm3_seed", "xgb_seed", "hist_seed", "xgb_op", "lgbm_k", "lgbm2_k", "cat_k", "xgb_k", "hist_k", "xgb_top"]
df = rawDF.withColumn('features', array([col(c) for c in cols1]))

In [11]:
df.printSchema()

root
 |-- lgbmcpu: double (nullable = true)
 |-- lgbmgpu: double (nullable = true)
 |-- xgb: double (nullable = true)
 |-- catgpu: double (nullable = true)
 |-- catcpu: double (nullable = true)
 |-- target: integer (nullable = true)
 |-- histcpu: double (nullable = true)
 |-- lgbmseed: double (nullable = true)
 |-- lgbmseed2: double (nullable = true)
 |-- lgbmseed3: double (nullable = true)
 |-- lgbmseed4: double (nullable = true)
 |-- voting_5cat: double (nullable = true)
 |-- voting_7cat: double (nullable = true)
 |-- voting_5xgb: double (nullable = true)
 |-- catseed: double (nullable = true)
 |-- lgbmgpu_seed: double (nullable = true)
 |-- lgbm1_seed: double (nullable = true)
 |-- lgbm2_seed: double (nullable = true)
 |-- lgbm3_seed: double (nullable = true)
 |-- xgb_seed: double (nullable = true)
 |-- hist_seed: double (nullable = true)
 |-- xgb_op: double (nullable = true)
 |-- lgbm_k: double (nullable = true)
 |-- lgbm2_k: double (nullable = true)
 |-- cat_k: double (nullable = 

In [12]:
dataset1 = df.select("features","target").withColumnRenamed("target","label")

In [13]:
dataset1.printSchema()

root
 |-- features: array (nullable = false)
 |    |-- element: double (containsNull = true)
 |-- label: integer (nullable = true)



In [14]:
dataset1.write.parquet("./train.parquet")

In [ ]:
dataset1.write.csv("./train_array.csv")#AnalysisException: 'CSV data source does not support array<double> data type